In [3]:
import yaml
import os
import pandas as pd
import streamlit as st
import pymysql

In [ ]:
df_list=[]
data_path='D:/Anto/Project/Project2/Dataset'
month_path=['2023-10','2023-11','2023-12','2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11']
date_path = [str(i) for i in range(1, 32)]
for i in month_path:
    for j in date_path:
        file_path=f"{data_path}/{i}/{i}-{j}_05-30-00.yaml"
        if not os.path.exists(file_path):
            continue
        with open(file_path, "r") as yaml_file:
            data = yaml.safe_load(yaml_file)  # Load YAML data
            
            # Handle different YAML structures
            if isinstance(data, list):  # List of dictionaries
                df = pd.DataFrame(data)
            elif isinstance(data, dict):  # Single dictionary (convert to list)
                df = pd.DataFrame([data])
            else:
                print(f"Skipping unsupported YAML format in: {file_path}")
                continue
            
            df_list.append(df)

# Combine all DataFrames into a single DataFrame
if df_list:
    final_df = pd.concat(df_list, ignore_index=False)
    final_df['date'] = pd.to_datetime(final_df['date']).dt.strftime("%y-%m-%d")
    final_df.to_csv("D:/Anto/Project/Project2/stock.csv", index=False)  # Save to a CSV file
    print("CSV file successfully created!")
else:
    print("No valid YAML files found!")

final_df

In [ ]:
#create 50 csv files ticker wise

#Defines the directory where the output CSV files will be stored
output_path = "D:/Anto/Project/Project2/CSV_Files"
os.makedirs(output_path, exist_ok=True)  # Ensure directory exists

nifty50=list(final_df['Ticker'].unique())
for k in nifty50:
    data=final_df.loc[final_df['Ticker']==k].copy()
    if not data.empty:
        output_file = f'{output_path}/{k}.csv'  # Output file path
        data.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")
    else:
        print(f"No data found for {k}, skipping...")

In [ ]:
#1.Volatility Analysis:

import pandas as pd

df=final_df

# Calculate daily return
df['daily_return'] = (df['close'] - df['open']) / df['open']

# Calculate standard deviation per ticker and rename column
volatility = df.groupby('Ticker')['daily_return'].std().reset_index()
volatility.rename(columns={'daily_return': 'std_dev'}, inplace=True)

# top 10 most volatile stocks
volatility_top_10 = volatility.nlargest(10, 'std_dev')

# Export as CSV
volatility_top_10.to_csv("D:/Anto/Project/Project2/Analysis/1.Volatility_Analysis.csv", index=False)

print("Volatility analysis data exported successfully!")

In [ ]:
#2.Cumulative Return Over Time:

import pandas as pd
df=final_df

df['daily_return'] = (df['close'] - df['open']) / df['open']

# Calculate cumulative return for each ticker
df['cumulative_return'] = (1 + df['daily_return']).groupby(df['Ticker']).cumprod() - 1

# Get the final cumulative return for each ticker (last row of each group)
final_cumulative_return = df.groupby('Ticker')['cumulative_return'].last()

# Sort by cumulative return and get top 5 stocks
top_5_stocks = final_cumulative_return.sort_values(ascending=False).head(5)


# Export the top 5 stocks to a CSV file
top_5_stocks.to_csv("D:/Anto/Project/Project2/Analysis/2.Cumulative_Return.csv")

print("Cumulative Return data exported successfully!")


In [ ]:
#3.Sector-wise Performance:

import pandas as pd
stock_df=final_df

#Reads the CSV file that contains sector-wise classification of stocks
sector_df=pd.read_csv(r'D:/Anto/Project/Project2/Dataset/Sector_data.csv')

#Calculates yearly return for each stock
stock_df['Yearly_Return'] = (stock_df['close'] - stock_df['open']) / stock_df['open']

#Merges stock_df and sector_df using the "Ticker" column as the key
merged_df = pd.merge(stock_df, sector_df, on='Ticker', how='left')

#Groups the data by sector and calculates the average Yearly Return for each sector
sector_returns = merged_df.groupby('sector')['Yearly_Return'].mean()

#Exports the sector-wise average yearly returns into a CSV file
sector_returns.to_csv("D:/Anto/Project/Project2/Analysis/3.Sector_Performance.csv")

print("sector wise performance data exported successfully!")


In [ ]:
#4.Stock Price Correlation:

import pandas as pd
df=final_df

#Calculates daily return for each stock 
df['daily_return'] = (df['close'] - df['open']) / df['open']

#Converts the DataFrame into a pivot table
pivot_df = df.pivot(index="date", columns="Ticker", values="daily_return")

#Renames specific stock tickers that contain special characters ,which are not accepted in SQL
pivot_df.rename(columns={"M&M": "MandM", "BAJAJ-AUTO": "BAJAJ_AUTO"}, inplace=True)

#Calculate correlation between stock daily returns.
correlation_df = pivot_df.corr()

#Saves the correlation data to a CSV file
correlation_df.to_csv("D:/Anto/Project/Project2/Analysis/4.Stock_Price_Correlation.csv")

print("Stock price correlation data exported successfully!")

In [ ]:
#5.Top 5 Gainers and Losers (Month-wise):

import pandas as pd
df=final_df

# Calculate monthly average open and close prices
monthly_avg = df.groupby(['Ticker', 'month']).agg({'open': 'mean', 'close': 'mean'}).reset_index()

# Calculate monthly return percentage
monthly_avg['Monthly_Return'] = (monthly_avg['close'] - monthly_avg['open']) / monthly_avg['open'] * 100

#Saves the data to a CSV file
monthly_avg.to_csv("D:/Anto/Project/Project2/Analysis/5.monthly_avg_return.csv",index=False)

print("monthly_avg_return data exported successfully!")